In [1]:
import numpy as np
import pandas as pd
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import metrics

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
raw_traininput = pd.read_csv('../data/train_input.csv')
raw_trainoutput = pd.read_csv('../data/train_output.csv')

raw_testinput = pd.read_csv('../data/test_input.csv')

traininput_size = raw_traininput.shape[0]
testinput_size = raw_testinput.shape[0]

In [3]:
import re

def clean_data(data):
    data_size = data.shape[0]
    tag_regex = '<.*?>|\n'
    
    cleaned_data = pd.DataFrame([ re.sub(tag_regex, '', data['conversation'][i]) \
                    for i in range(data_size) ], columns = {('conversation')})
    
    return cleaned_data

In [17]:
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

clean_traininput = raw_traininput
print clean_traininput.shape

cvec = CountVectorizer(analyzer='word', 
                       stop_words = 'english',
                       ngram_range = (1,5))

classification = Pipeline([('vectorizer', cvec),
                           ('transformer', TfidfTransformer()),
                           ('classifier', LinearSVC(C=3, loss='hinge'))])

train_data, test_data, train_labels, test_labels = train_test_split(clean_traininput['conversation'], 
                                                                    raw_trainoutput['category'], 
                                                                    test_size=0.2, 
                                                                    random_state=105)

classification = classification.fit(train_data, train_labels)
predicted = classification.predict(test_data)
print np.mean(predicted == test_labels)
print metrics.classification_report(test_labels, predicted)
print metrics.confusion_matrix(test_labels, predicted)

(165000, 2)
0.966666666667
             precision    recall  f1-score   support

     hockey       0.98      0.98      0.98      4170
     movies       0.98      0.99      0.99      4510
        nba       0.99      0.97      0.98      3696
       news       0.94      0.91      0.92      4256
        nfl       0.98      0.98      0.98      3978
   politics       0.93      0.95      0.94      3943
     soccer       0.99      0.99      0.99      4299
  worldnews       0.95      0.96      0.95      4148

avg / total       0.97      0.97      0.97     33000

[[4086   15   24    3   24    1   14    3]
 [   5 4474    1   11    1    6    3    9]
 [  40   18 3575    8   31    3   18    3]
 [   1   17    3 3879   13  180    2  161]
 [  27    5   18    7 3911    3    6    1]
 [   2    1    1  148    1 3751    1   38]
 [  16    5    5    4    8    2 4248   11]
 [   2    8    0   85    0   67   10 3976]]


In [18]:
clean_testinput = raw_testinput
print clean_testinput.shape

cvec2 = CountVectorizer(analyzer='word', 
                       stop_words = 'english',
                       ngram_range = (1,5))

classification2 = Pipeline([('vectorizer', cvec2),
                           ('transformer', TfidfTransformer()),
                           ('classifier', LinearSVC(C=3, loss='hinge'))])

classification2 = classification2.fit(clean_traininput['conversation'], raw_trainoutput['category'])
predicted2 = classification2.predict(clean_testinput['conversation'])

(53218, 2)


In [19]:
result = pd.DataFrame({'category' : predicted2})
result.to_csv('../data/test_predict_LSVM_5gramC3.csv', index = True, header = True, index_label = 'id')

In [7]:
clean_testinput.shape

(53218, 2)

In [1]:
0.97159*53218

51706.07662